In [ ]:
import enum
from dataclasses import dataclass
import math
import random

# import pandas as pd
from matplotlib import pyplot

class ActionType(enum.IntEnum):
    SELF_USE = 0,
    FEED_IN = 1,
    BACKUP = 2,
    CHARGE = 3

@dataclass
class Action:
    type: ActionType
    min_soc: float
    max_soc: float

@dataclass
class TimeSegment:
    generation: float
    consumption: float
    feed_in_tariff: float
    import_tariff: float

@dataclass
class RunResult:
    import_cost: float = 0
    feed_in_cost: float = 0
    battery_level: float = 0
    battery_cumulative_charge: float = 0
    max_solar_battery: float = 0

SEGMENT_LENGTH_HOURS = 1
BATTERY_CAPACITY = 4.2
BATTERY_CHARGE_AMOUNT_PER_SEGMENT = 3 * SEGMENT_LENGTH_HOURS
EXPORT_LIMIT_PER_SEGMENT = 9999
AC_TO_DC_EFFICIENCY = 0.95
DC_TO_AC_EFFICIENCY = 0.8
EXPORT_LIMIT_PER_SEGMENT_DC = EXPORT_LIMIT_PER_SEGMENT / DC_TO_AC_EFFICIENCY

# Lowest that we can choose to discharge the battery to
MIN_SOC_PERMITTED_PERCENT = 10
SOC_STEP_PERCENT = 10

num_runs = 0

def run(segments: list[TimeSegment], actions: list[Action | None], initial_battery: float, debug: bool = False) -> RunResult:
    global num_runs
    num_runs += 1
    battery_level = initial_battery
    result = RunResult()
    action = Action(ActionType.SELF_USE, min_soc=0.2, max_soc=1.0)

    # Debug
    battery_levels = []
    imports = []
    exports = []

    for i, (segment, action_change) in enumerate(zip(segments, actions)):
        if action_change is not None:
            action = action_change
        solar_to_battery = 0
        solar_to_grid = 0
        battery_to_load = 0
        grid_to_load = 0
        grid_to_battery_ac = 0
        grid_to_battery_dc = 0

        if action.type == ActionType.SELF_USE:
            # If generation can cover consumption, excess goes into battery. Else excess comes from battery if available
            if segment.generation > segment.consumption / DC_TO_AC_EFFICIENCY:
                excess_solar_dc = segment.generation - segment.consumption / DC_TO_AC_EFFICIENCY
                solar_to_battery = max(0, min(BATTERY_CAPACITY * action.max_soc - battery_level, excess_solar_dc))
                solar_to_grid = min(EXPORT_LIMIT_PER_SEGMENT_DC, excess_solar_dc - solar_to_battery) * DC_TO_AC_EFFICIENCY
            else:
                required_energy_ac = segment.consumption - segment.generation * DC_TO_AC_EFFICIENCY
                battery_to_load = max(0, min(battery_level - BATTERY_CAPACITY * action.min_soc, required_energy_ac / DC_TO_AC_EFFICIENCY))
                grid_to_load = required_energy_ac - battery_to_load * DC_TO_AC_EFFICIENCY
        elif action.type == ActionType.FEED_IN:
            # Generation goes to grid rather than battery (up to export limit), but consumption still draws from battery
            if segment.generation > segment.consumption / DC_TO_AC_EFFICIENCY:
                excess_solar_dc = segment.generation - segment.consumption / DC_TO_AC_EFFICIENCY
                solar_to_grid_dc = min(EXPORT_LIMIT_PER_SEGMENT_DC, excess_solar_dc)
                solar_to_grid = solar_to_grid_dc * DC_TO_AC_EFFICIENCY
                solar_to_battery = min(BATTERY_CAPACITY, excess_solar_dc - solar_to_grid_dc)
            else:
                required_energy_ac = segment.consumption - segment.generation * DC_TO_AC_EFFICIENCY
                battery_to_load = max(0, min(battery_level - BATTERY_CAPACITY * action.min_soc, required_energy_ac / DC_TO_AC_EFFICIENCY))
                grid_to_load = required_energy_ac - battery_to_load * DC_TO_AC_EFFICIENCY
        elif action.type == ActionType.BACKUP:
            # PV goes first to batteries, then to house, with excess being exported
            solar_to_battery = max(0, min(BATTERY_CAPACITY * action.max_soc - battery_level, segment.generation))
            excess_solar_ac = (segment.generation - solar_to_battery) * DC_TO_AC_EFFICIENCY
            if excess_solar_ac > segment.consumption:
                solar_to_grid = min(EXPORT_LIMIT_PER_SEGMENT, excess_solar_ac - segment.consumption)
            else:
                grid_to_load = segment.consumption - excess_solar_ac
        elif action.type == ActionType.CHARGE:
            # I don't actually know, but... I assume that solar replaces grid up to the charge rate
            solar_to_battery = max(0, min(BATTERY_CAPACITY * action.max_soc - battery_level, segment.generation))
            grid_to_battery_dc = max(0, min(BATTERY_CAPACITY * action.max_soc - battery_level - solar_to_battery, BATTERY_CHARGE_AMOUNT_PER_SEGMENT))
            grid_to_battery_ac = grid_to_battery_dc / AC_TO_DC_EFFICIENCY
            excess_solar_ac = (segment.generation - solar_to_battery) * DC_TO_AC_EFFICIENCY
            # Doesn't consider inverter limits
            if excess_solar_ac > segment.consumption:
                solar_to_grid = min(EXPORT_LIMIT_PER_SEGMENT, excess_solar_ac - segment.consumption)
            else:
                grid_to_load = segment.consumption - excess_solar_ac

        battery_change = solar_to_battery + grid_to_battery_dc - battery_to_load
        battery_level += battery_change
        assert(battery_level >= 0)

        # TODO: Not currently working
        result.battery_cumulative_charge += battery_level
        result.feed_in_cost += solar_to_grid * segment.feed_in_tariff
        result.import_cost += (grid_to_load + grid_to_battery_ac) * segment.import_tariff
        if battery_change > 0 and battery_level >= BATTERY_CAPACITY and action.type != ActionType.CHARGE and battery_level > result.max_solar_battery:
            result.max_solar_battery = battery_level
            # print(f"Filled the battery at {i}")
            

        if debug:
            imports.append(grid_to_load + grid_to_battery_ac)
            exports.append(solar_to_grid)
            battery_levels.append(battery_level)

    if debug:
        pyplot.plot(range(0, len(battery_levels)), battery_levels, marker='o', label='batt')
        pyplot.plot(range(0, len(segments)), [x.consumption for x in segments], marker='x', label='cons')
        pyplot.plot(range(0, len(segments)), [x.generation for x in segments], marker='+', label='gen')
        pyplot.plot(range(0, len(imports)), imports, marker='<', label='gen')
        pyplot.plot(range(0, len(exports)), exports, marker='>', label='gen')
        pyplot.show()

    result.battery_level = battery_level
    return result

def shotgun_hillclimb(segments: list[TimeSegment], initial_battery: float):

    def score_result(result: RunResult) -> float:
        return round(result.feed_in_cost) - round(result.import_cost) #+ round(result.battery_level * 20)

    def is_better(x: RunResult, y: RunResult) -> bool:
        if x.max_solar_battery != y.max_solar_battery:
            return x.max_solar_battery > y.max_solar_battery
        x_score = score_result(x)
        y_score = score_result(y)
        # if x_score != y_score:
        return x_score > y_score
        # Reward filling the battery at some point during the day

        # If they score equal, choose the one with the larger amount of charge held in the batter for the longest
        # This inventivises runs which charge the battery earlier, which reduces risk
        # TODO: However, this isn't currently working
        # print(f"Bleh. {x_score}, {y_score}, {x.battery_cumulative_charge}, {y.battery_cumulative_charge}")
        # return x.battery_cumulative_charge > y.battery_cumulative_charge

    action_type_set = [ActionType.SELF_USE, ActionType.CHARGE]
    best_result_ever: RunResult | None = None
    best_actions_ever: list[Action] = []
    for _ in range(20):
        # actions = [Action.SELF_USE] * 2 + [Action.BACKUP] * 3 + [Action.SELF_USE] * 11 + [Action.FEED_IN] * 3 + [Action.SELF_USE] * 5
        # Keeping a fair number of "Do last action" seems to make it easier for it to find solutions which only work
        # if you consistently do the same thing a lot.
        actions = [None] * len(segments) 
        # actions = [Action(ActionType.SELF_USE, 0.2, 1.0) for _ in range(len(segments))]
        for _ in range(10):
            action_type = random.choice(action_type_set)
            # No point having a min soc when charging
            min_soc_percent = MIN_SOC_PERMITTED_PERCENT if action_type == ActionType.CHARGE else random.randrange(MIN_SOC_PERMITTED_PERCENT, 101, SOC_STEP_PERCENT)
            actions[random.randint(0, len(segments) / 2 - 1)] = Action(
                action_type,
                min_soc = min_soc_percent / 100.0,
                max_soc = random.randrange(min_soc_percent, 101, SOC_STEP_PERCENT) / 100.0,
            )
        print(actions)
        # actions = [random.choice(actions_set) for _ in range(len(segments))]
        # actions = [Action.CHARGE] * len(segments)
        best_actions = actions.copy()
        best_result = run(segments, actions, initial_battery)
        # print(f"Starting best score: {best_score}")
        while True:
            found_better = False
            # We evaluate each of the possible changes, and see which one has the greatest effect
            best_improved_result = best_result
            best_improved_actions: list[Action] | None = None
            # Shuffling these means we choose a random action from those with the best score
            slots = list(range(len(actions)))
            random.shuffle(slots)
            for slot in slots:
                for new_action_type in action_type_set:
                    # TODO: Efficiency
                    # if actions[slot] is not None and actions[slot] == new_action_type:
                    #     continue
                    for new_min_soc_percent in [MIN_SOC_PERMITTED_PERCENT] if new_action_type == ActionType.CHARGE or segments[slot].generation > segments[slot].consumption / DC_TO_AC_EFFICIENCY else range(MIN_SOC_PERMITTED_PERCENT, 101, SOC_STEP_PERCENT):
                        for new_max_soc_percent in range(new_min_soc_percent, 101, SOC_STEP_PERCENT) if new_action_type == ActionType.CHARGE or segments[slot].generation > segments[slot].consumption  / DC_TO_AC_EFFICIENCY else [100]:
                        # for new_max_soc_percent in range(new_min_soc_percent, 100, 20):
                            prev_action = actions[slot]
                            actions[slot] = Action(new_action_type, min_soc=new_min_soc_percent / 100, max_soc=new_max_soc_percent / 100)
                            new_result = run(segments, actions, initial_battery)
                            if is_better(new_result, best_improved_result):
                                # print(f"Changing {slot} from {prev_action} to {new_action} gives {new_score} >= {best_improved_score}")
                                found_better = True
                                best_improved_result = new_result
                                best_improved_actions = actions.copy()
                            actions[slot] = prev_action
            if found_better:
                # Did we find an improvement? Keep going
                best_result = best_improved_result
                best_actions = actions = best_improved_actions
            else:
                # No? We've reached a local maximum
                break
        if best_result_ever is None or is_better(best_result, best_result_ever):
            best_result_ever = best_result
            best_actions_ever = best_actions
    
    # Do a final pass. This time, try and simplify: if changing a slot to a "lower" action doesn't hurt the score, do it
    # Also get rid of Nones

    # run(segments, best_actions_ever, initial_battery, debug=True)
    # best_actions_ever[2] = Action(ActionType.CHARGE, 0.1, 0.45)
    # # # best_actions_ever[3] = Action(ActionType.SELF_USE, 1.0, 1.0)
    # # print(best_result_ever)
    # best_result_ever = run(segments, best_actions_ever, initial_battery, debug=True)
    # print(best_result_ever)
    
    prev_action = Action(ActionType.SELF_USE, min_soc=MIN_SOC_PERMITTED_PERCENT/100, max_soc=1.0)
    for slot in range(len(actions)):
        if best_actions_ever[slot] is None:
            best_actions_ever[slot] = prev_action
        else:
            prev_action = best_actions_ever[slot]
        for action_type in action_type_set:
            if int(action_type) >= int(best_actions_ever[slot].type):
                break
            prev_action_type = best_actions_ever[slot].type
            best_actions_ever[slot].type = action_type
            new_result = run(segments, best_actions_ever, initial_battery)
            # If the old result was better, go back to it and continue. Otherwise go for the new result
            if not is_better(best_result_ever, new_result):
                break
            # assert(new_score < best_score_ever)
            best_actions_ever[slot].type = prev_action_type

        # If we can decrease the min soc without hurting the score, do that
        prev_min_soc = best_actions_ever[slot].min_soc
        for min_soc_percent in range(MIN_SOC_PERMITTED_PERCENT, round(prev_min_soc * 100), SOC_STEP_PERCENT):
            best_actions_ever[slot].min_soc = min_soc_percent / 100
            new_result = run(segments, best_actions_ever, initial_battery)
            if not is_better(best_result_ever, new_result):
                break
            best_actions_ever[slot].min_soc = prev_min_soc
        prev_max_soc = best_actions_ever[slot].max_soc

        # TODO: If we're using the max soc to prevent charging, then decrease it
        # If we're using the max soc to prevent charging, then decrease it to 0.1
        # If we can increase the max soc without hurting the score, do that 
        for max_soc_percent in range(100, round(prev_max_soc * 100), -SOC_STEP_PERCENT):
            best_actions_ever[slot].max_soc = max_soc_percent / 100
            new_result = run(segments, best_actions_ever, initial_battery)
            if not is_better(best_result_ever, new_result):
                break
            best_actions_ever[slot].max_soc = prev_max_soc


    for i, action in enumerate(best_actions_ever):
        print(f"{i}: {action}")



    print(best_result_ever)
    print(score_result(best_result_ever))
    run(segments, best_actions_ever, initial_battery, debug=True)

    global num_runs
    print(f"Number of runs: {num_runs}")





# df = pd.read_csv('load_power_hourly.csv', usecols=['start', 'mean'], parse_dates=['start'], index_col='start') 
# consumption = [x for x in df[21:(21+24)]['mean']]
consumption = [0.2, 0.4, 0.2, 0.2, 0.2, 0.31, 0.25, 0.41, 0.32, 0.32, 0.29, 0.4, 0.57, 2.53, 0.82, 0.32, 0.32, 0.22, 0.11, 0.45, 0.2, 0.1, 0.2, 0.2]
generation = [0, 0, 0, 0, 0, 0, 0.05, 0.15, 0.72, 2.04, 2.33, 2.27, 2.35, 2.1, 1.94, 1.26, 0.75, 1.26, 0.11, 0.06, 0, 0, 0, 0]
# generation[0:9] = [0] * 11
# generation[16:] = [0] * 8
import_tariff = [30.72] * 2 + [18.43] * 3 + [30.72] * 11 + [43.01] * 3 +  [30.72] * 5
feed_in_tariff = [19.72] * 2 + [7.43] * 3 + [19.72] * 11 + [32.01] * 3 + [19.72] * 5

# consumption = [x for x in consumption for _ in (0, 1)]
# generation = [x for x in generation for _ in (0, 1)]
# feed_in_tariff = [15] * 24
# import_tariff = [22.1, 20.5, 19.3, 17.9, 18.7, 18.0, 17.8, 17.6, 
# 18.3, 17.1, 20.9, 22.1, 22.3, 24.3, 24.4, 29.0, 
# 24.7, 26.8, 23.6, 21.9, 20.1, 18.0, 18.1, 17.2, 
# 19.6, 16.5, 18.7, 15.9, 18.3, 16.0, 20.3, 20.3, 
# 33.5, 34.8, 34.5, 38.7, 38.6, 38.7, 26.1, 24.7, 
# 26.4, 25.8, 24.7, 20.9, 17.9, 8.8, 8.8, 8.8]

segments = [TimeSegment(generation=g*0.8, consumption=c*1.4, feed_in_tariff=f, import_tariff=i) for g, c, f, i in zip(generation, consumption, feed_in_tariff, import_tariff)]
# segments = segments + segments
shotgun_hillclimb(segments, 2)

# actions = [Action.SELF_USE] * 2 + [Action.BACKUP] * 3 + [Action.SELF_USE] * 11 + [Action.FEED_IN] * 3 + [Action.SELF_USE] * 5

# result = run(segments, actions, 2.1, True)
# print(result)